# Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded, dependencies will be auto-updated.

In [ ]:
# Free tier
# symlink_to_notebooks = True                          # Enables the creation of symlinks back to /notebooks/
# model_storage_dir = '/tmp/stable-diffusion/models'   # Where the models will be downloaded to.
# repo_storage_dir = '/notebooks'                      # Where the repository will be downloaded to.

# Paid Tier
# symlink_to_notebooks = False
# model_storage_dir = '/storage/models'
# repo_storage_dir = '/notebooks'


# Other optional settings
# You don't have to change these if you don't want to

activate_xformers = True                              # Enables the xformers optimizations using pre-built wheels.
                                                       # Setting to True will automatically set up your environment/machine for xformers. 

link_novelai_anime_vae = False                         # Enables the linking of animevae.pt to each of the NovelAI models.
                                                       # Set to True if you've downloaded both the NovelAI models and hypernetworks.

download_scripts = True                               # Download custom scripts? Only reason why you would leave it disabled is because it may
                                                       # take a while to complete.

activate_deepdanbooru = False                          # Enable and install DeepDanbooru -> https://github.com/KichangKim/DeepDanbooru

activate_medvram = False                                # Enable medvram option.
                                                       # These are model optimizations which will reduce VRAM usage at the expense of some speed.
                                                       # Set to False if you have a lot of VRAM.

disable_pickle_check = True                           # Disable the automatic check for unexpected data in model files.
                                                       # Leave this set to False unless you have a reason to disable the check.

gradio_port = False                                    # Launch Gradio on a specific port. Set to False to let Gradio choose a port.
                                                       # This disables online Gradio app mode and you will only be able to access it on your local network.

gradio_auth = False                                    # Enable gradio_auth and insecure-extension-access option.
                                                       # Set to "me:password" to enable.

search_paperspace_datasets = True                      # Enable searching for checkpoints in /datasets to link to the webui

ui_theme = None                                        # Set the WEB UI theme. Values can be None (default) or 'dark'.

insecure_extension_access = True                      # Force enable extensions without a password.
                                                       # If you don't set a password, anyone can install and run arbitrary code on your machine!
                                                       # Instead, use gradio_auth which will automatically enable extensions when set.


# ===================================================================================================
# Save variables to Jupiter's temp storage so we can access it even if the kernel restarts.
%store symlink_to_notebooks model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae download_scripts activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth search_paperspace_datasets ui_theme insecure_extension_access

## Clone the WebUI repository

In [ ]:
import os
# You'll see this little code block at the beginning of every cell. It makes sure you have ran the first block that defines your settings.
try:
    %store -r symlink_to_notebooks model_storage_dir repo_storage_dir
    test = [symlink_to_notebooks, model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

%cd /notebooks/

def delete_broken_symlinks(path):
    # make sure to pass this function a path without a trailing slash
    for file in os.listdir(path):
        if os.path.islink(f'{path}/{file}') and not os.path.exists(os.readlink(f'{path}/{file}')):
            print(f'Symlink broken, removing: {file}')
            os.unlink(f'{path}/{file}')

def update_repo_if_not_exists(path, repo_clone_url, pre=None):
    if pre is not None:
        pre()    
    if not os.path.exists(path):
        !git clone "{repo_clone_url}" "{path}"
    else:
        print(f'{repo_clone_url.split("/")[-1]} already downloaded, updating...')
        !cd "{path}" && git pull # no % so we don't interfere with the main process

def init_free():
    if (symlink_to_notebooks and repo_storage_dir != '/notebooks'):
        delete_broken_symlinks('/notebooks/') # remove broken symlinks since it might have been installed in a non-persistent directory
        if not os.path.exists(repo_storage_dir):
            !mkdir -p "{repo_storage_dir}"
            !ln -s "{repo_storage_dir}" /notebooks/
            !ls -la /notebooks/stable-diffusion
update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui', 'https://github.com/AUTOMATIC1111/stable-diffusion-webui', init_free)

## Install requirements and download repositories

In [ ]:
try:
    %store -r symlink_to_notebooks model_storage_dir repo_storage_dir activate_xformers download_scripts activate_deepdanbooru
    test = [symlink_to_notebooks, model_storage_dir, repo_storage_dir, activate_xformers, download_scripts, activate_deepdanbooru]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

%cd "{repo_storage_dir}/stable-diffusion-webui"

!pip install --upgrade pip

import os

# Import launch.py which will automatically run the install script but not launch the WebUI.
# They require a few specific external git repo commits so we have to do it their way. 
import launch
launch.prepare_environment()

# The installer isn't installing deepdanbooru right now so we'll do it manually
if activate_deepdanbooru:
    !pip install "git+https://github.com/KichangKim/DeepDanbooru.git@edf73df4cdaeea2cf00e9ac08bd8a9026b7a7b26#egg=deepdanbooru[tensorflow]" # tensorflow==2.10.0 tensorflow-io==0.27.0 flatbuffers==1.12

# latent-diffusion is a requirement but launch.py isn't downloading it so we'll do it manually.
if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion'):
    !git clone https://github.com/crowsonkb/k-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/k-diffusion"
    !git clone https://github.com/Hafiidz/latent-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion"

# For things in this notebook
!pip install requests
!pip install gdown

# Download popular custom scripts. This is basically remote code execution so be careful.
# See https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Custom-Scripts
if download_scripts:
    import shutil
    import requests
    !pip install moviepy==1.0.3
    !apt update
    !apt install -y potrace python3-tk

    def download_file_dir(url, output_dir):
        # output_dir must have a trailing slash
        local_filename = url.split('/')[-1]
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(f'{output_dir}{local_filename}', 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        return local_filename
    def do_script_download(scripts_list, domain, path):
        for item in scripts_list:
            download_file_dir(f'https://{domain}/{item}', path)
            print(f'{item.split("/")[-1]} downloaded...')

    do_script_download([
        'GRMrGecko/stable-diffusion-webui-automatic/advanced_matrix/scripts/advanced_prompt_matrix.py',
        'dfaker/stable-diffusion-webui-cv2-external-masking-script/main/external_masking.py',
        'memes-forever/Stable-diffusion-webui-video/main/videos.py',
        'yownas/seed_travel/main/scripts/seed_travel.py',
        'Animator-Anon/Animator/main/animation.py',
        'Filarius/stable-diffusion-webui/master/scripts/vid2vid.py',
        'GeorgLegato/Txt2Vectorgraphics/main/txt2vectorgfx.py',
        'yownas/shift-attention/main/scripts/shift_attention.py',
        'DiceOwl/StableDiffusionStuff/main/loopback_superimpose.py',
        'Engineer-of-Stuff/stable-diffusion-paperspace/main/lfs/save_steps.py',
        'Pfaeff/sd-web-ui-scripts/main/moisaic.py'
    ], 'raw.githubusercontent.com', f'{repo_storage_dir}/stable-diffusion-webui/scripts/')

    do_script_download([
        'dfaker/f88aa62e3a14b559fe4e5f6b345db664/raw/791dabfa0ab26399aa2635bcbc1cf6267aa4ffc2/alternate_sampler_noise_schedules.py',
        'camenduru/9ec5f8141db9902e375967e93250860f/raw/c1a03eb447548adbef1858c0e69d3567a390d2f4/run_n_times.py'
    ], 'gist.githubusercontent.com', f'{repo_storage_dir}/stable-diffusion-webui/scripts/')

    # Download and set up txt2img2img
    update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root', 'https://github.com/ThereforeGames/txt2img2img.git')
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/scripts" "{repo_storage_dir}/stable-diffusion-webui"
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/txt2img2img" "{repo_storage_dir}/stable-diffusion-webui"
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/venv" "{repo_storage_dir}/stable-diffusion-webui"

    # Download and set up txt2mask
    update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui/txt2mask', 'https://github.com/ThereforeGames/txt2mask.git')
    !echo "Copying txt2mask..."
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2mask/repositories/clipseg" "{repo_storage_dir}/stable-diffusion-webui/repositories"
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2mask/scripts/" "{repo_storage_dir}/stable-diffusion-webui/"

    # Install the dynamic-prompts/wildcard script
    # !git clone https://github.com/adieyal/sd-dynamic-prompting/ extensions/dynamic-prompts

if activate_xformers:
    print('Installing xformers...')
    import subprocess
    def download_release(url):
        binary = 'xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl' # have to save the binary as a specific name that pip likes
        tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
        !wget "{url}" -O "{tmp_dir}/{binary}"
        return os.path.join(tmp_dir, binary)

    # Set up pip packages
    !pip uninstall -y torch  torchvision torchaudio # Remove existing pytorch install.
    !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113 # Install pytorch for cuda 11.3
    s = subprocess.getoutput('nvidia-smi')
    if 'A4000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A4000-Oct-28-2022/a4000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'A5000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A5000-Nov-1-2022/a5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'A6000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A6000-Nov-1-2022/a6000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'P5000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/P5000-Nov-1-2022/p5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'RTX 4000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-4000-Nov-1-2022/rtx4000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'RTX 5000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-5000-Nov-1-2022/rtx5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'A100' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'M4000' in s:
        print('xformers for M4000 hasn\'t been built yet.')
        # xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    else:
        print('GPU not matched to xformers binary so a one-size-fits-all binary was installed. If you have any issues, please build xformers using the Tools block below.')
        xformers_whl = download_release('https://raw.githubusercontent.com/Cyberes/xformers-compiled/main/various/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl')
    !pip install --force-reinstall "{xformers_whl}"

# Make sure your models storage directory exists
!mkdir -p "{model_storage_dir}/hypernetworks"
!mkdir -p "{model_storage_dir}/vae"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks"

# Link the output folders to /notebooks/outputs
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/log/images"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/outputs"
!ln -s "{repo_storage_dir}/stable-diffusion-webui/outputs" /notebooks/
!ln -s "{repo_storage_dir}/stable-diffusion-webui/log" "{repo_storage_dir}/stable-diffusion-webui/outputs"

# Link /storage/ to /notebooks/
!ln -s /storage/ /notebooks/

!echo -e "\n===================================\nDone! If you're seeing this the process has exited successfully.\n"

## Clean up and restart the kernel

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

# Get some storage back
!pip cache purge
!cd "{model_storage_dir}" && rm *.aria2
!apt remove --purge -y aria2 p7zip-full
!apt autoremove --purge -y
!apt clean

# Restart the kernel
import os
os.kill(os.getpid(), 9)

## Link the models directory

Create symlinks. The file will be stored in the models storage directory and linked to where the WebUI expects the files to be.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir link_novelai_anime_vae search_paperspace_datasets
    test = [model_storage_dir, repo_storage_dir, link_novelai_anime_vae, search_paperspace_datasets]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

import os
import glob
import subprocess
import re
from pathlib import Path
import sys

if not os.path.isdir(model_storage_dir):
    print('Your model storage directory does not exist:', model_storage_dir)
    sys.exit(1)

webui_model_path = Path(repo_storage_dir, 'stable-diffusion-webui/models')
webui_sd_models = Path(webui_model_path, 'Stable-diffusion')
webui_hypernetworks = Path(webui_model_path, 'hypernetworks')
webui_vaes = Path(webui_model_path, 'VAE')

def delete_broken_symlinks(dir):
    deleted = False
    for file in os.listdir(dir):
        path = f'{dir}/{file}'
        if os.path.islink(path) and not os.path.exists(os.readlink(path)):
            print(f'Symlink broken, removing: {file}')
            os.unlink(path)
            deleted = True
    if deleted:
        print('')
        
def create_symlink(source, dest):
    if os.path.isdir(dest):
        dest = Path(dest, os.path.basename(source))
    if not os.path.exists(dest):
        !ln -s "{source}" "{dest}"
    print(source, '->', Path(dest).absolute())

# Check for broken symlinks and remove them
print('Removing broken symlinks...')
delete_broken_symlinks(webui_sd_models)
delete_broken_symlinks(webui_hypernetworks)
delete_broken_symlinks(webui_vaes)

# Link .ckpt files (recursive)
print('\nLinking .ckpt files...')
for file in glob.glob(f'{model_storage_dir}/**/*.ckpt', recursive=True):
    if not Path(file).parent.parts[-1] in ['hypernetworks', 'vae'] :
        if not os.path.exists(file):
            print('New model:', os.path.basename(file))
        create_symlink(file, webui_sd_models)

# Link config yaml files
print('\nLinking config .yaml files...')
for file in glob.glob(f'{model_storage_dir}/**/*.yaml', recursive=True):
    create_symlink(file, webui_sd_models)

# Link hypernetworks
print('\nLinking hypernetworks...')
source_hypernetworks = Path(model_storage_dir, 'hypernetworks')
if os.path.isdir(source_hypernetworks):
    for file in os.listdir(source_hypernetworks):
        create_symlink(Path(source_hypernetworks, file), webui_hypernetworks)
else:
    print('Hypernetwork storage directory not found:', source_hypernetworks)

# Link VAEs
print('\nLinking VAEs...')
source_vaes = Path(model_storage_dir, 'vae')
if os.path.isdir(source_vaes):
    for file in os.listdir(source_vaes):
        create_symlink(Path(source_vaes, file), webui_vaes)
else:
    print('VAE storage directory not found:', source_vaes)

# Link the NovelAI files for each of the NovelAI models
print('\nLinking NovelAI files for each of the NovelAI models...')
for model in glob.glob(f'{model_storage_dir}/novelai-*.ckpt'):
    yaml = model.replace('.ckpt', '.yaml')
    if os.path.exists(yaml):
        # if not os.path.exists(yaml):
        print(f'New NovelAI model config: {yaml}')
        create_symlink(yaml, webui_sd_models)

if link_novelai_anime_vae:
    print('\nLinking NovelAI anime VAE...')
    for model in glob.glob('novelai-*.ckpt'):
        if os.path.exists(Path(model_storage_dir, 'hypernetworks/animevae.pt')):
            vae = model.replace('.ckpt', '.vae.pt')
            if not os.path.exists(webui_vaes):
                print(f'Linking NovelAI {vae} and {model}')
            create_symlink(Path(model_storage_dir, 'hypernetworks/animevae.pt'), webui_vaes)
        else:
            print(f'{model_storage_dir}/hypernetworks/animevae.pt NOT FOUND')
            

if search_paperspace_datasets:
    if os.path.isdir('/datasets'):
        print('\nSearching /datasets')
        for file in glob.glob(f'/datasets/**/*.ckpt', recursive=True):
            create_symlink(file, webui_sd_models)
        for file in glob.glob(f'/datasets/**/*.yaml', recursive=True):
            create_symlink(file, webui_sd_models)
    else:
        print('\nNo datasets mounted!')

# Launch the WebUI

Run this block to launch the WebUI. You will get a link to nnn.gradio.app, that's your WebUI. Follow it.

See [shared.py](https://github.com/AUTOMATIC1111/stable-diffusion-webui/blob/master/modules/shared.py#L22) to view the code for the launch args. There's a lot of good info in there about exactly what the args do. If you aren't a programmer, [here's the wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings).

#### Troubleshooting
- If you have any issues, try restarting the kernel.
- `EOFError: Ran out of input` probably means you ran out of storage space and the model `.ckpt` file wasn't downloaded completely. Try cleaning up your files. There are some helpful scripts in the Tools section below.
- `The file may be malicious, so the program is not going to read it` means the program encountered unexpected data in the model file (the technical term is "pickle"). Merging models can cause this. You can disable this feature by setting `disable_pickle_check` to True in the settings block.
- Try updating your notebook using the block in the Tools section below.
- If you're still having issues, delete `stable-diffusion-webui` and reinstall.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir activate_xformers activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth ui_theme insecure_extension_access
    test = [model_storage_dir, repo_storage_dir, activate_xformers, activate_deepdanbooru, activate_medvram, disable_pickle_check, gradio_port, gradio_auth, ui_theme, insecure_extension_access]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

%cd "{repo_storage_dir}/stable-diffusion-webui"

# Code to set the options you want as defined in the very first block
x_arg = '--xformers' if activate_xformers else ''
dd_arg = '--deepdanbooru' if activate_deepdanbooru else ''
mvram_arg = '--medvram' if activate_medvram else ''
pickled = '--disable-safe-unpickle' if disable_pickle_check else ''
port = f'--port {gradio_port}' if gradio_port else '--share'
auth = f'--gradio-auth {gradio_auth} --enable-insecure-extension-access' if gradio_auth else ''
theme = f'--theme {ui_theme}' if ui_theme else ''
insecure_extension_access = '--enable-insecure-extension-access' if insecure_extension_access else ''

# Launch args go below:
!python webui.py {x_arg} {dd_arg} {mvram_arg} {pickled} {port} {auth} {theme} --gradio-debug